In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysa.emd as emddev
import pysa.eemd as eemddev
import pysa.visualization as plotter
import pysa.utils as utils
import pysa.nhht as nhht
import copy
from multiprocessing import Pool
from timeit import default_timer as timer

from scipy import signal
import scipy
import os
from scipy import fft
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
%matplotlib inline
test = 1
plt.rcParams['figure.figsize'] = (12,8)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def IMF_filter(signal):
    max_modes = 15
    ensembles = 100
    ensembles_per_process = 10
    max_siftings = 200
    end_time = 10
    sample_freq = 250

    max_data = max(signal)
    min_data = min(signal)
    
    imfs = emddev.emd(signal, min_data, max_data, max_modes, max_siftings)
    
    imf1 = utils.reverse_normalization(imfs[0], min_data, max_data, len(signal))
    imf2 = utils.reverse_normalization(imfs[1], min_data, max_data, len(signal))
    residue = utils.reverse_normalization(imfs[-1], min_data, max_data, len(signal))
    return signal - imf1 - imf2 - residue

In [3]:
# Output imf x data
def get_IMF(signal, max_modes = 15):
    ensembles = 100
    ensembles_per_process = 10
    max_siftings = 200
    end_time = 10
    sample_freq = 250

    max_data = max(signal)
    min_data = min(signal)
    
    return emddev.emd(signal, min_data, max_data, max_modes, max_siftings)

In [4]:
def window_average(x, window = 5):
    avg_x = np.zeros(len(x) // window)
    for i in range(0, (len(x) // window)):
        w_step = i * window
        avg_x[i] = np.average(x[w_step:(w_step + 4)])
    return avg_x

In [5]:
# Takes in an array of form measurement x data x sensor
# Return array of form measurement x data x sensor
def filter_signals(data):
    f = np.zeros(data.shape)
    # Calculate all IMFs for all sensors
    for m, measurement in enumerate(data):
        for s, sensor in enumerate(measurement.T):
            f[m, :,s] = IMF_filter(sensor)
    return f

In [6]:
def calculate_IMFs(data, num_imfs=4):
    # Calculate all IMFs for all sensors
    list_imfs = np.zeros((data.shape[0], data.shape[2], num_imfs, data.shape[1]))
    for m, measurement in enumerate(data):
        for s, sensor in enumerate(measurement.T):
            imfs = get_IMF(sensor)
            num_cols = 4
            if imfs.shape[0] < 4:
                num_cols = imfs.shape[0]
            list_imfs[m, s, :num_cols, :] = imfs[0:num_cols] 
    return list_imfs

In [7]:
def calculate_HHT(imfs):
    err_idx = np.array([])
    frequencies = np.zeros(imfs.shape)
    amplitudes = np.zeros(imfs.shape)
    for m, measurement in enumerate(imfs):
        for s, sensor in enumerate(measurement):
                try:
                    f, a = nhht.nhht(sensor, 250)
                except:
                    print("Error on measurement " + str(m) + "sensor " + str(s))
                    err_idx = np.append(err_idx, m)
                frequencies[m,s,:,:] = f
                amplitudes[m,s,:,:] = a
    return frequencies, amplitudes, err_idx

In [8]:
def ERP(amplitudes, divider=3, initial=1, cutoff=1, fs=250):
    
    a = np.square(amplitudes)
    # Calculate mean before event
    r_i = a[:, (initial * fs):(divider * fs)].mean(axis=1)
    a_j = np.zeros(a[:, (divider * fs):-(cutoff * fs)].shape)
    for r, row in enumerate(a[:, (divider * fs):-(cutoff * fs)]):
        a_j[r, :] = [100 * ((a_j_t - r_i[r]) / r_i[r]) for a_j_t in row]
    # Calculate mean after event
    #a_j = a[:, (divider * fs):-(cutoff * fs)].mean(axis=1)
    # Calculate event related potential for given amplitudes
    #erp =  np.mean(100 * ((a_j - r_i) / r_i))
    erp = a_j.mean(axis=1)
    
    return erp

In [9]:
def calculate_ERP(amplitudes):
    erps = np.zeros((amplitudes.shape[0], amplitudes.shape[1], 4))
    for m, measurement in enumerate(amplitudes):
        for a, amplitude in enumerate(measurement):
                erps[m,a,:] = ERP(amplitude)
    return erps

In [10]:
def load_all_signals(folder):
    path = folder
    files = []
    count = 0
    for i in os.listdir(path):
        files.append(i)
    data = []
    for file in files:
        df = pd.read_csv(os.path.join(path, file), index_col=0)
        data.append(df.as_matrix())
    return np.asarray(data)

In [11]:
def signal_to_erp(signal):
    err_idx = np.array([])
    #print("Calculating fd")
    fd = filter_signals(signal)
    fd = fd[:,125:-125,:]
    print("Calculating imfs")
    imfs = calculate_IMFs_alt(signal)
    #imfs = imfs[:,:,:,100:-100]
    print("Calculating erps")
    erps = np.zeros((imfs.shape[0] - 3, imfs.shape[1], imfs.shape[2]))
    for m, measurement in enumerate(imfs[2:-1,:,:]):
        for s, sensor in enumerate(measurement):
                try:
                    _, a = nhht.nhht(sensor, 250)
                except:
                    err_idx = np.append(err_idx, m)
                erps[m,s,:] = ERP(a[:, 125:-125])
    return imfs, erps, err_idx

In [12]:
def classify_SVM(erps):
    train = np.vstack(tuple(erps))
    X, Y = train[:, 0:-1], train[:, -1].astype(int)
    clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(kernel='rbf'))
    scores = cross_val_score(clf, X, Y, cv=3, scoring='accuracy')
    print("Accuracy: ", scores.mean())

In [13]:
from pprint import pprint
def grid_search_SVM(erps):
    train = np.vstack(tuple(erps))
    np.random.shuffle(train)
    X_train, Y_train = train[:, 0:-1], train[:, -1].astype(int)
    
    tuned_parameters2 = [{'kernel': ['rbf'], 'gamma': [0.0001,0.001,0.01,1.0,2],
                         'C': [1,2,4,6,8,10]},
                        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1, 1e-2, 1e-3, 1e-4, 1e-5],
                         'C': [0.1, 1, 10, 100, 1000, 10000]},
                        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    scaler = StandardScaler()
    scaler.fit(X_train)

    grid = GridSearchCV(estimator=svm.SVC(C=1), param_grid=tuned_parameters, cv=10, scoring='accuracy', n_jobs=-1) 
    grid.fit(scaler.transform(X_train), Y_train)

    # View the accuracy score
    print('Best score for data1:', grid.best_score_)
    # View the best parameters for the model found using grid search
    print('Best C:',grid.best_estimator_.C) 
    print('Best Kernel:', grid.best_estimator_.kernel)
    print('Best Gamma:', grid.best_estimator_.gamma)
    pprint(grid.grid_scores_)

In [14]:
path = "exp_data/joachim"

In [15]:
right = load_all_signals(path + "/right")
neutral = load_all_signals(path + "/neutral")
left = load_all_signals(path + "/left")

In [16]:
right.shape

(50, 1500, 8)

In [17]:
#print("Calculating fd")
p = Pool(3)
fd1, fd2, fd3 = p.map(filter_signals, [neutral[:,125:-125,:], right[:,125:-125,:], left[:,125:-125,:]])
p.close()
p.join()

In [18]:
p = Pool(3)
imfs1, imfs2, imfs3 = p.map(calculate_IMFs, [fd1, fd2, fd3])
p.close()
p.join()

In [19]:
p = Pool(3)
hht1, hht2, hht3 = p.map(calculate_HHT, [imfs1[:,:,:,125:-125], imfs2[:,:,:,125:-125], imfs3[:,:,:,125:-125]])
p.close()
p.join()
freq1, ampl1, ei1 = hht1
freq2, ampl2, ei2 = hht2
freq3, ampl3, ei3 = hht3

In [20]:
print(fd1.shape, imfs1.shape, freq1.shape, ampl1.shape)

(50, 1250, 8) (50, 8, 4, 1250) (50, 8, 4, 1000) (50, 8, 4, 1000)


In [21]:
np.save(path + '/fd1', fd1)
np.save(path + '/fd2', fd2)
np.save(path + '/fd3', fd3)

np.save(path + '/imfs1', imfs1)
np.save(path + '/imfs2', imfs2)
np.save(path + '/imfs3', imfs3)

np.save(path + '/freq1', freq1)
np.save(path + '/freq2', freq2)
np.save(path + '/freq3', freq3)

np.save(path + '/ampl1', ampl1)
np.save(path + '/ampl2', ampl2)
np.save(path + '/ampl3', ampl3)